In [ ]:
# Установка зависимостей
!pip install numpy matplotlib mpmath scipy plotly

In [ ]:
# Импорт библиотек и определение функций
import numpy as np
import matplotlib.pyplot as plt
from mpmath import zeta
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, clear_output
pio.renderers.default = 'iframe'  # Для корректного отображения в Binder

# Параметры
T = 20  # Интервал времени
dt = 1.0  # Шаг для ускорения
N = int(2 * T / dt)  # Количество точек (N = 40)
t = np.linspace(-T, T, N)  # Массив времени
t_vals = np.linspace(-T, T, 30)  # Для шага 5

# Пример мнимых частей нетривиальных нулей (из LMFDB)
gamma_n = [14.1347, 21.0220]  # Первые два нуля

# Функции из статьи
def psi(s, u, epsilon):
    # Вычисление функции ψ(s, u)
    denom = (s - 0.5 - 1j*u)**2 * (1 - s - 0.5 - 1j*u)**2 + epsilon**2
    return 1 / denom

def zeta_squared(t):
    # Модуль квадрата дзета-функции на критической линии
    return abs(complex(zeta(0.5 + 1j*t)))**2

def exp_term(s, epsilon):
    # Экспоненциальный множитель для регуляризации
    u = np.linspace(-T, T, N)
    integrand = [complex(psi(s, ui, epsilon) * zeta_squared(ui)) for ui in u]
    return np.exp(-np.trapezoid(integrand, u))

def kernel(ti, tj, epsilon):
    # Ядро оператора \(\hat{H}\)
    s = 0.5 + 1j*(ti - tj)
    return complex(exp_term(s, epsilon) * zeta(s))

def compute_kernel(epsilon):
    # Вычисление матрицы ядра
    K = np.zeros((N, N), dtype=complex)
    for i in range(N):
        if i % 10 == 0:  # Выводим прогресс каждые 10 итераций
            print(f"Вычисление ядра: {i}/{N}")
        for j in range(N):
            K[i, j] = kernel(t[i], t[j], epsilon)
    return K

def compute_trace(K):
    # След оператора \(\hat{H}\)
    return np.trace(K)

def selberg_sum(t, gamma_n):
    # Сумма косинусов для формулы Сельберга
    return sum(np.cos(t * gamma) for gamma in gamma_n)

In [ ]:
# Визуализация процесса доказательства гипотезы Римана через ССТАФ
epsilon_values = [1e-5, 5e-5, 1e-4]  # Значения epsilon для перебора

for epsilon in epsilon_values:
    print(f"Визуализация для ε = {epsilon}")

    # Шаг 1: Регуляризация \(\mathcal{R}\zeta(s)\)
    print("Шаг 1: Регуляризация \(\mathcal{R}\zeta(s)\)")
    t_imag = np.linspace(-20, 20, 50)
    s_vals = 0.5 + 1j * t_imag
    R_zeta = [exp_term(s, epsilon) * complex(zeta(s)) for s in s_vals]
    fig_R = go.Figure()
    fig_R.add_trace(go.Scatter(x=t_imag, y=np.abs(R_zeta), mode='lines', name='|\(\mathcal{R}\zeta(\frac{1}{2} + it)|'))
    fig_R.update_layout(
        title='Модуль \(\mathcal{R}\zeta(s)\) на критической линии',
        xaxis_title='t (мнимая часть)',
        yaxis_title='|\(\mathcal{R}\zeta\)|'
    )
    display(fig_R)

    # Шаг 2: Ядро оператора \(\hat{H}\)
    print("Шаг 2: Ядро оператора \(\hat{H}\")
    K = compute_kernel(epsilon)
    print("Минимальное значение |K(t, t')|:", np.abs(K).min())
    print("Максимальное значение |K(t, t')|:", np.abs(K).max())
    fig1 = go.Figure(data=go.Heatmap(z=np.abs(K), x=t, y=t, colorscale='Hot'))
    fig1.update_layout(
        title='Модуль ядра |K(t, t\')|',
        xaxis_title='t\'',
        yaxis_title='t',
        annotations=[
            dict(x=gamma_n[0], y=gamma_n[0], xref="x", yref="y", text="γ₁", showarrow=True, arrowhead=2),
            dict(x=gamma_n[1], y=gamma_n[1], xref="x", yref="y", text="γ₂", showarrow=True, arrowhead=2)
        ]
    )
    display(fig1)

    # Шаг 3: Собственные значения
    print("Шаг 3: Собственные значения и нули")
    eigenvalues = np.linalg.eigvalsh(K)
    print(f"Количество собственных значений: {len(eigenvalues)} (ожидается N = {N})")
    fig2, ax2 = plt.subplots()
    ax2.scatter(np.arange(len(eigenvalues)), np.abs(eigenvalues), s=10)
    for gamma in gamma_n:
        ax2.axhline(y=gamma, color='r', linestyle='--', alpha=0.5)
    ax2.set_title('Собственные значения \(\hat{H}\)')
    ax2.set_xlabel('Индекс')
    ax2.set_ylabel('|λ|')
    ax2.set_yscale('log')
    ax2.set_ylim(1e-6, 1e2)
    plt.show()

    # Шаг 4: Спектральная симметрия
    print("Шаг 4: Спектральная симметрия (подтверждение гипотезы Римана)")
    sigma_vals = np.linspace(0, 1, 30)
    gamma = 14.1347  # Пример мнимой части
    integrals_rho = [np.trapezoid([complex(psi(sigma + 1j*gamma, u, epsilon) * zeta_squared(u)) for u in t], t) for sigma in sigma_vals]
    integrals_1_minus_rho = [np.trapezoid([complex(psi(1 - sigma + 1j*gamma, u, epsilon) * zeta_squared(u)) for u in t], t) for sigma in sigma_vals]
    fig_sym = go.Figure()
    fig_sym.add_trace(go.Scatter(x=sigma_vals, y=integrals_rho, mode='lines', name='Интеграл для \(\rho\)'))
    fig_sym.add_trace(go.Scatter(x=sigma_vals, y=integrals_1_minus_rho, mode='lines', name='Интеграл для \(1-\rho\)'))
    fig_sym.add_vline(x=0.5, line_dash="dash", line_color="red", annotation_text="Критическая линия")
    fig_sym.update_layout(
        title='Симметрия интегралов',
        xaxis_title='σ (действительная часть)',
        yaxis_title='Значение интеграла'
    )
    display(fig_sym)

    # Шаг 5: Анализ следа
    print("Шаг 5: Анализ следа")
    trace_vals = [complex(compute_trace(K)) for _ in t_vals]
    selberg_vals = [selberg_sum(ti, gamma_n) for ti in t_vals]
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=t_vals, y=trace_vals, mode='lines', name='След (\(\hat{H}\))'))
    fig3.add_trace(go.Scatter(x=t_vals, y=selberg_vals, mode='lines', name='Σ cos(t γ_n)'))
    fig3.update_layout(
        title='Анализ следа',
        xaxis_title='t',
        yaxis_title='Значение'
    )
    display(fig3)

    # Очистка вывода перед следующим значением epsilon
    input("Нажмите Enter для продолжения с новым ε...")
    clear_output(wait=True)